In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from MEDIC.secrets import controller

In [3]:
total_df = pd.read_csv('./total_raw.csv').drop('Unnamed: 0', axis=1)
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104791 entries, 0 to 104790
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CountryCode            104791 non-null  object 
 1   date                   104791 non-null  object 
 2   covid_confirmed        104702 non-null  float64
 3   covid_deaths           104702 non-null  float64
 4   covid_recovered        104702 non-null  float64
 5   test_cumulTotal        41225 non-null   float64
 6   test_7dayMA            45215 non-null   float64
 7   test_shortPosRate      42982 non-null   float64
 8   test_shortTestperCase  42474 non-null   float64
 9   vac_first              12066 non-null   float64
 10  vac_second             10207 non-null   float64
 11  vac_daily              17665 non-null   float64
 12  mob_retailRecreation   54393 non-null   float64
 13  mob_groceryPharmacy    54379 non-null   float64
 14  mob_parks              54128 non-nul

In [4]:
cur_df = total_df.loc[total_df['CountryCode'] == 'AFG']
set(cur_df.loc[:, 'test_cumulTotal'].isnull()) == {True}

True

In [5]:
remove_list = list()
for country in sorted(set(total_df['CountryCode'])):
    cur_df = total_df.loc[total_df['CountryCode'] == country]
    if set(cur_df.loc[:, 'test_cumulTotal'].isnull()) == {True}\
        or set(cur_df.loc[:, 'test_7dayMA'].isnull()) == {True}\
        or set(cur_df.loc[:, 'test_shortPosRate'].isnull()) == {True}\
        or set(cur_df.loc[:, 'test_shortTestperCase'].isnull()) == {True}\
        or set(cur_df.loc[:, 'vac_first'].isnull()) == {True}\
        or set(cur_df.loc[:, 'vac_second'].isnull()) == {True}\
        or set(cur_df.loc[:, 'vac_daily'].isnull()) == {True}\
        or set(cur_df.loc[:, 'mob_retailRecreation'].isnull()) == {True}\
        or set(cur_df.loc[:, 'mob_groceryPharmacy'].isnull()) == {True}\
        or set(cur_df.loc[:, 'mob_parks'].isnull()) == {True}\
        or set(cur_df.loc[:, 'mob_transit'].isnull()) == {True}\
        or set(cur_df.loc[:, 'mob_work'].isnull()) == {True}\
        or set(cur_df.loc[:, 'mob_resident'].isnull()) == {True}:
        
        remove_list.append(country)

In [6]:
total_df = total_df[~total_df['CountryCode'].isin(remove_list)]

In [7]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53028 entries, 2156 to 104771
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CountryCode            53028 non-null  object 
 1   date                   53028 non-null  object 
 2   covid_confirmed        52958 non-null  float64
 3   covid_deaths           52958 non-null  float64
 4   covid_recovered        52958 non-null  float64
 5   test_cumulTotal        39771 non-null  float64
 6   test_7dayMA            43641 non-null  float64
 7   test_shortPosRate      42071 non-null  float64
 8   test_shortTestperCase  41618 non-null  float64
 9   vac_first              11257 non-null  float64
 10  vac_second             9616 non-null   float64
 11  vac_daily              16355 non-null  float64
 12  mob_retailRecreation   50258 non-null  float64
 13  mob_groceryPharmacy    50256 non-null  float64
 14  mob_parks              50099 non-null  float64
 15

In [8]:
for country in sorted(set(total_df['CountryCode'])):
    total_df.loc[total_df['CountryCode'] == country,'covid_confirmed'] = \
        total_df.loc[total_df['CountryCode'] == country,'covid_confirmed'].fillna(method='ffill').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'covid_deaths'] = \
        total_df.loc[total_df['CountryCode'] == country,'covid_deaths'].fillna(method='ffill').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'covid_recovered'] = \
        total_df.loc[total_df['CountryCode'] == country,'covid_recovered'].fillna(method='ffill').fillna(method='bfill')
    
    total_df.loc[total_df['CountryCode'] == country,'test_cumulTotal'] = \
        total_df.loc[total_df['CountryCode'] == country,'test_cumulTotal'].fillna(method='ffill').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'test_7dayMA'] = \
        total_df.loc[total_df['CountryCode'] == country,'test_7dayMA'].fillna(method='ffill').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'test_shortPosRate'] = \
        total_df.loc[total_df['CountryCode'] == country,'test_shortPosRate'].fillna(method='ffill').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'test_shortTestperCase'] = \
        total_df.loc[total_df['CountryCode'] == country,'test_shortTestperCase'].fillna(method='ffill').fillna(method='bfill')
    
    total_df.loc[total_df['CountryCode'] == country,'vac_first'] = \
        total_df.loc[total_df['CountryCode'] == country,'vac_first'].fillna(method='ffill').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'vac_second'] = \
        total_df.loc[total_df['CountryCode'] == country,'vac_second'].fillna(method='ffill').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'vac_daily'] = \
        total_df.loc[total_df['CountryCode'] == country,'vac_daily'].fillna(method='bfill').fillna(method='ffill')
    
    total_df.loc[total_df['CountryCode'] == country,'mob_retailRecreation'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_retailRecreation'].interpolate(method='linear').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'mob_groceryPharmacy'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_groceryPharmacy'].interpolate(method='linear').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'mob_parks'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_parks'].interpolate(method='linear').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'mob_transit'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_transit'].interpolate(method='linear').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'mob_work'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_work'].interpolate(method='linear').fillna(method='bfill')
    total_df.loc[total_df['CountryCode'] == country,'mob_resident'] = \
        total_df.loc[total_df['CountryCode'] == country,'mob_resident'].interpolate(method='linear').fillna(method='bfill')

In [9]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53028 entries, 2156 to 104771
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CountryCode            53028 non-null  object 
 1   date                   53028 non-null  object 
 2   covid_confirmed        53028 non-null  float64
 3   covid_deaths           53028 non-null  float64
 4   covid_recovered        53028 non-null  float64
 5   test_cumulTotal        53028 non-null  float64
 6   test_7dayMA            53028 non-null  float64
 7   test_shortPosRate      53028 non-null  float64
 8   test_shortTestperCase  53028 non-null  float64
 9   vac_first              53028 non-null  float64
 10  vac_second             53028 non-null  float64
 11  vac_daily              53028 non-null  float64
 12  mob_retailRecreation   53028 non-null  float64
 13  mob_groceryPharmacy    53028 non-null  float64
 14  mob_parks              53028 non-null  float64
 15

In [10]:
print(set(total_df[total_df['covid_confirmed'].isnull()]['CountryCode']))
print(set(total_df[total_df['covid_deaths'].isnull()]['CountryCode']))
print(set(total_df[total_df['covid_recovered'].isnull()]['CountryCode']))

set()
set()
set()


In [11]:
len(set(total_df['date']))

561